# Exploratory Analysis of Spatial Data

3. create interactive visualization in bokeh
5. create interactive mask connection

### To-Do:
* COnnection between plots is not working - Datasources differ
* data manipulation necesserby, but inside or outside of functions?

### Note:
* .patches does not support lasso or box select (https://github.com/bokeh/bokeh/issues/2325),
    they do support selection by clicking on a "patch"

## Imports

In [1]:
%matplotlib inline 

import matplotlib.pyplot as plt
import pysal as ps
import libpysal.api as lp
import numpy as np
import pandas as pd

import geopandas as gpd
import os
import splot
import splot.plot
import bokeh

In [2]:
link_to_data = ps.examples.get_path('columbus.shp')
df = gpd.read_file(link_to_data)
#df.head()

In [3]:
y = df['HOVAL'].values
w = lp.Queen.from_dataframe(df)
w.transform = 'r'

In [4]:
import esda
moran_loc = esda.moran.Moran_Local(y, w)

## Exploring Local Autocorrelation interactively with splot

In [5]:
from collections import OrderedDict
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool

from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper, CategoricalColorMapper

import geopandas as gpd
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

## Utility Functions

In [6]:
def add_legend(fig, labels, colors):
    """
    Add a legend to a figure given legend labels & colors.
    
    Parameters
    ----------
    fig : Bokeh Figure instance
        Figure instance labels should be generated for.
    labels : list of str
        Labels to use as legend entries.
    colors : Bokeh Palette instance
        Palette instance containing colours of choice.
    """
    # add labels to figure (workaround, legend with geojsondatasource doesn't work,
    # see https://github.com/bokeh/bokeh/issues/5904)
    for label, color in zip(labels, colors):
        fig.patches(xs=[], ys=[], fill_color=color, legend=label)

In [7]:
def bin_values_choropleth(attribute_values, method='quantiles',
                          k=5):
    '''
    Create bins based on different classification methods.
    Needed for legend labels and Choropleth coloring.

    Parameters
    ----------
    attribute_values : array or geopandas.series instance
        Array containing relevant attribute values.
    method : str
        Classification method to be used. Options supported:
        * 'quantiles' (default)
        * 'fisher-jenks'
        * 'equal-interval'
    k : int
        Number of bins, assigning values to. Default k=5

    Returns
    -------
    bin_values : mapclassify instance
        Object containing bin ids for each observation (.yb),
        upper bounds of each class (.bins), number of classes (.k)
        and number of onservations falling in each class (.counts)    
    '''
    import mapclassify.api as classify
    
    classifiers = {
        'quantiles': classify.Quantiles,
        'fisher-jenks': classify.Fisher_Jenks,
        'equal-interval': classify.Equal_Interval
    }
    
    bin_values = classifiers[method](attribute_values, k)

    return bin_values

In [8]:
def bin_labels_choropleth(df, attribute_values, method='quantiles', k=5):
    '''
    Create labels for each bin in the legend
    
    Parameters
    ----------
    df : Geopandas dataframe
        Dataframe containign relevant shapes and attribute values.
    attribute_values : array or geopandas.series instance
        Array containing relevant attribute values.
    method : str, optional
        Classification method to be used. Options supported:
        * 'quantiles' (default)
        * 'fisher-jenks'
        * 'equal-interval'
    k : int, optional
        Number of bins, assigning values to. Default k=5
    
    Returns
    -------
    bin_labels : list of str
        List of label for each bin.
    '''    
    # Retrieve bin values from bin_values_choropleth()
    bin_values = bin_values_choropleth(attribute_values, method=method, k=k)
    
    # Extract bin ids (.yb) and upper bounds for each class (.bins)
    yb = bin_values.yb
    bins = bin_values.bins

    # Create bin labels
    bin_edges = [attribute_values.min()] + bins.tolist()  # for use in legend
    bin_labels = []
    for i in range(k):
        bin_labels.append('{:1.1f}-{:1.1f}'.format(bin_edges[i], bin_edges[i+1]))
        
    # Add labels (which are the labels printed in the legend) to each row of df
    labels = np.array([bin_labels[c] for c in yb])
    df['labels'] = [str(l) for l in labels]
    
    return bin_labels

## Choropleth map in Bokeh

In [9]:
def plot_choropleth_bokeh(df, attribute, title=None, plot_width=500,
                        plot_height=500, method='quantiles',
                        k=5, reverse_colors=False, tools=''):
    '''
    Plot Choropleth colored according to attribute
    
    Parameters
    ----------
    df : Geopandas dataframe
        Dataframe containign relevant shapes and attribute values.
    attribute : str
        Name of column containing attribute values of interest.
    title : str, optional
        Title of map. Default title=None
    plot_width : int, optional
        Width dimension of the figure in screen units/ pixels.
        Default = 500
    plot_height : int, optional
        Height dimension of the figure in screen units/ pixels.
        Default = 500
    method : str, optional
        Classification method to be used. Options supported:
        * 'quantiles' (default)
        * 'fisher-jenks'
        * 'equal-interval'
    k : int, optional
        Number of bins, assigning values to. Default k=5
    reverse_colors: boolean
        Reverses the color palette to show lightest colors for
        lowest values. Default reverse_colors=False
    
    Returns
    -------
    fig : Bokeh Figure instance
        Figure of Choropleth
    '''
    # We're adding columns, do that on a copy rather than on the users' input
    df = df.copy()
    
    # Extract attribute values from df
    attribute_values = df[attribute].values
    
    # Create bin labels with bin_labels_choropleth()
    bin_labels = bin_labels_choropleth(df, attribute_values, method, k)
    
    # Initialize GeoJSONDataSource
    geo_source = GeoJSONDataSource(geojson=df.to_json())
    
    from bokeh import palettes 
    colors = palettes.Blues[k]
    if reverse_colors is True:
        colors.reverse()  # lightest color for lowest values

    # Create figure
    fig = figure(title=title, plot_width=plot_width, plot_height=plot_height, tools=tools)
    fig.patches('xs', 'ys', fill_alpha=0.7, 
              fill_color={'field': 'labels',
                          'transform': CategoricalColorMapper(palette=colors,
                                                              factors=bin_labels)},
              line_color='white', line_width=0.5, source=geo_source)
    
    # add legend with add_legend()
    add_legend(fig, bin_labels, colors)
    
    # change layout
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    fig.axis.visible = None
    
    return fig

In [11]:
TOOLS = "tap,help"
fig = plot_choropleth_bokeh(df, 'HOVAL', title='columbus', reverse_colors=True, tools=TOOLS)
show(fig)

## LISA cluster map in Bokeh

In [12]:
def mask_local_auto(moran_loc, df=None, p=0.5):
    '''
    Create Mask for coloration and labeling of local spatial autocorrelation
    
    Parameters
    ----------
    moran_loc : esda.moran.Moran_Local instance
        values of Moran's I Global Autocorrelation Statistic
    df : geopandas dataframe instance, optional
        If given, assign df['labels'] per row.  Note that `df` will be
        modified, so calling functions should use a copy of the user
        provided `df`.
    p : float
        The p-value threshold for significance. Points will
        be colored by significance.
    
    Returns
    -------
    cluster_labels : list of str
        List of labels - ['ns', 'HH', 'LH', 'LL', 'HL']
    colors5 : list of str
        List of colours - ['lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors : array of str
        Array containing coloration for each input value/ shape.
    '''
    # create a mask for local spatial autocorrelation
    sig = 1 * (moran_loc.p_sim < p)
    HH = 1 * (sig * moran_loc.q==1)
    LL = 3 * (sig * moran_loc.q==3)
    LH = 2 * (sig * moran_loc.q==2)
    HL = 4 * (sig * moran_loc.q==4)

    cluster = HH + LL + LH + HL
    cluster_labels = ['ns', 'HH', 'LH', 'LL', 'HL']
    labels = [cluster_labels[i] for i in cluster]
    # create a new column with label info
    if df is not None:
        df['labels'] = np.array(labels)

    colors5 = ['lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors = [colors5[i] for i in cluster]
    
    return cluster_labels, colors5, colors, labels
    

In [13]:
def lisa_cluster_bokeh(moran_loc, df, p=0.05, title=None, plot_width=500,
                          plot_height=500, tools=''): 
    '''
    Lisa Cluster map, coloured by local spatial autocorrelation
    
    Parameters
    ----------
    moran_loc : esda.moran.Moran_Local instance
        values of Moran's I Global Autocorrelation Statistic
    df : geopandas dataframe instance
        In mask_local_auto(), assign df['labels'] per row. Note that 
        ``df`` will be modified, so calling functions uses a copy of
        the user provided ``df``.
    p : float, optional
        The p-value threshold for significance. Points will
        be colored by significance.
    title : str, optional
        Title of map. Default title=None
    plot_width : int, optional
        Width dimension of the figure in screen units/ pixels.
        Default = 500
    plot_height : int, optional
        Height dimension of the figure in screen units/ pixels.
        Default = 500
    
    Returns
    -------
    fig : Bokeh figure instance
        Figure of LISA cluster map, colored by local spatial autocorrelation
    '''
    # We're adding columns, do that on a copy rather than on the users' input
    df = df.copy()
    
    # add cluster_labels and colors5 in mask_local_auto
    cluster_labels, colors5, _, _ = mask_local_auto(moran_loc, df=df, p=0.05)
    
    # Load the Tools
    #TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    
    # load df into bokeh data source
    geo_source = GeoJSONDataSource(geojson=df.to_json())

    # Create figure
    fig = figure(title=title, toolbar_location='right',
          plot_width=plot_width, plot_height=plot_height, tools=tools)
    fig.patches('xs', 'ys', fill_alpha=0.8, 
              fill_color={'field': 'labels', 'transform': CategoricalColorMapper(palette=colors5,
                                                                                 factors=cluster_labels)}, 
              line_color='white', line_width=0.5, source=geo_source)
    
    # add legend with add_legend()
    add_legend(fig, cluster_labels, colors5)
    
    # change layout
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    fig.axis.visible = None

    return fig

TOOLS = "tap,reset,help"
fig = lisa_cluster_bokeh(moran_loc, df, p=0.05, tools=TOOLS)
show(fig)

In [14]:
from bokeh.models import Span
    
def mplot_bokeh(moran_loc, p=None, plot_width=500, plot_height=500, tools=''):    
    lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
    fit = ps.spreg.OLS(moran_loc.z[:, None], lag[:,None])
    
    if p is not None:
        if not isinstance(moran_loc, esda.moran.Moran_Local):
            raise ValueError("`moran_loc` is not a Moran_Local instance")
    
        _, _, colors, _ = mask_local_auto(moran_loc, p=0.05)
    
    else:
        colors = 'black'
    
    #Tools
    #TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    # Vertical line
    vline = Span(location=0, dimension='height', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    # Horizontal line
    hline = Span(location=0, dimension='width', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    
    # Create figure
    fig = figure(title="Moran Scatterplot", x_axis_label='Response', y_axis_label='Spatial Lag',
                 toolbar_location='left', plot_width=plot_width, plot_height=plot_height, tools=tools)
    fig.scatter(moran_loc.z, lag, color=colors, size=8, fill_alpha=.6)
    fig.renderers.extend([vline, hline])
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    fig.line(lag, fit.predy.flatten(), line_width = 2) # fit line
    
    return fig

In [15]:
fig = mplot_bokeh(moran_loc, p=0.05)
show(fig)

# Combined visualizations

Static visualization of Moran Scatterplot, LISA cluster map and choropleth map

In [17]:
from bokeh.layouts import gridplot

def three_plot_bokeh(moran_loc, df, attribute, p=0.05, plot_width=300, plot_height=300):
    
    TOOLS = "tap,reset,help"
    
    scatter = mplot_bokeh(moran_loc, p=p, plot_width=plot_width, plot_height=plot_height, tools=TOOLS)
    LISA = lisa_cluster_bokeh(moran_loc, df, p=p, plot_width=plot_width, plot_height=plot_height, tools=TOOLS)
    choro = plot_choropleth_bokeh(df, attribute, reverse_colors=True, plot_width=plot_width, plot_height=plot_height,
                                tools=TOOLS)
    
    fig = gridplot([[scatter, LISA, choro]])
    
    return fig

fig = three_plot_bokeh(moran_loc, df, 'HOVAL')
show(fig)

## Back-up code options:

In [ ]:
# not a good way
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i, shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j, member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

lons, lats = gpd_bokeh(df)

def filter_nans(lons, lats):
    newlons = []
    newlats = []
    for lon, lat in zip(lons, lats):
        ix = np.isnan(lon) & np.isnan(lat)
        newlons.append(np.array(lon)[~ix])
        newlats.append(np.array(lat)[~ix])
        
    return newlons, newlats
        
newlons, newlats = filter_nans(lons, lats)
#df['newlons'] = newlons
#df['newlats'] = newlats

Example for adding data to dataframe

df['lons'] = np.array(lons)
df['lats'] = np.array(lats)

lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
df['lag'] = np.array(lag)